In [1]:
from fastai.vision.all import *

In [2]:
path = untar_data(URLs.PETS)/'images'
fnames = get_image_files(path)
pat = r'/([^/]+)_\d+.*'
batch_tfms = [*aug_transforms(size=224, max_warp=0), Normalize.from_stats(*imagenet_stats)]
item_tfms = RandomResizedCrop(460, min_scale=0.75, ratio=(1.,1.))
bs=64

pets = DataBlock(
    blocks=(ImageBlock, CategoryBlock),
     get_items=get_image_files,
     splitter=RandomSplitter(),
     get_y=RegexLabeller(pat = r'/([^/]+)_\d+.*'),
     item_tfms=item_tfms,
     batch_tfms=batch_tfms
)
dls = pets.dataloaders(path, bs=bs)

In [3]:
learn = vision_learner(dls, "vit_tiny_patch16_224")

In [7]:
dls.c

In [17]:
learn.save("MyModel", with_opt=False)

In [18]:
from timm import create_model
class CustomTimmBody(nn.Module):
    """
    A small submodule to work with `timm` models more easily
    """
    def __init__(
        self, 
        model, 
        pretrained:bool=True, 
        cut=None, 
        n_in:int=3
    ):
        super().__init__()
        self.needs_pooling = model.default_cfg.get('pool_size', None)
        if cut is None:
            self.model = model
        else:
            self.model = custom_cut_model(model, cut)
    
    def forward(self, x): 
        if self.needs_pooling:
            return self.model.forward_features(x)
        else:
            return self.model(x)

In [25]:
create_model("vit_tiny_patch16_224", pretrained=True, num_classes=0, in_chans=3)

In [19]:
body = CustomTimmBody(
    create_model("vit_tiny_patch16_224", pretrained=True, num_classes=0, in_chans=3)
)
head = create_head(body.model.num_features, 37, pool=None)

In [20]:
net = nn.Sequential(body, head)

In [21]:
state = torch.load("models/MyModel.pth")

In [22]:
net.load_state_dict(state)